In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import os
import time

NUM_EPOCHS = 1500 # Number of training epochs
PH = 1

tf.set_random_seed(777)

def readData(filePath) :
    x_data = []
    y_data = []
    allList = []
    newPointx=[]
    newPointy=[]
    
    with open(filePath, 'r') as f:
        for line in f:
            allList.append(float(line))

    df = pd.Series(allList) #일차원 리스트를 pandas 데이터프레임화

    while True:
        for i in df[0:7]: 
            newPointx.append(float(i)) #데이터프레임 앞의 7개를 newPointx 리스트에 삽입
        newPointy.append(float(df[6 + (PH)])) #데이터프레임 그 다음(10번째)을 newPointy 리스트에 삽입

        x_data.append(newPointx) #x_data 리스트에 newPointx 리스트를 삽입 (x_data는 array of array가 됨)
        y_data.append(newPointy) #위와 동일

        newPointx=[] #다음 반복을 위해 newPointx,y를 빈 리스트로 초기화
        newPointy=[]
        df=df.shift(-1) #데이터프레임 왼 쪽으로 1칸 쉬프트
        if(math.isnan(df[6 + (PH)])): #만약 8번째 데이터프레임이 NaN(Not a Number) 라면 반복 중지
            break

    #배열 갯수 보정작업
    #데이터의 개수가 항상 8의 배수가 아니기 때문에 x_data의 마지막 원소 리스트가 항상 7개가 아닐수 있고
    #y_data의 마지막 원소 리스트가 항상 1개가 아닐 수 있기 때문에 
    #빈 칸들은 0으로 채워주기 위한 작업
    if(len(x_data[-1]) != 7):
       xSize = 5-len(x_data[-1])
       for i in range(xSize):
           x_data[-1].append(0.0)
    if(len(y_data[-1])!=1):
       y_data[-1].append(0.0)  
    
    data = [x_data, y_data]
    return data;

filename = os.listdir("sch")

train_data_name = []
test_data_name = []

for fn in filename:
    if fn.find("test") != -1:
        test_data_name.append(fn)
    elif fn.find("train") != -1:
        train_data_name.append(fn)


total_x_data = []
total_y_data = []        
        
train_x_data = []
train_y_data = []
for fn in train_data_name:
    x,y = readData("sch/"+str(fn))
    train_x_data.append(x)
    total_x_data.append(x)
    train_y_data.append(y)
    total_y_data.append(y)
    

test_x_data = []
test_y_data = []
for fn in test_data_name:
    x,y = readData("sch/"+str(fn))
    test_x_data.append(x)
    total_x_data.append(x)
    test_y_data.append(y)
    total_y_data.append(y)

#print(total_x_data)

In [2]:
x = tf.placeholder(tf.float32,shape=[None,7])
y = tf.placeholder(tf.float32,shape=[None,1])

w1 = tf.Variable(tf.random_uniform([7,15],minval = -1 , maxval = 1,dtype=tf.float32))
b1 = tf.Variable(tf.zeros([1,15]))

w2 = tf.Variable(tf.random_uniform([15,15],minval = -1,maxval =1,dtype=tf.float32))
b2 = tf.Variable(tf.zeros([1,15]))

w3 = tf.Variable(tf.random_uniform([15,1],minval = -1,maxval = 1,dtype=tf.float32))
b3 = tf.Variable(tf.zeros([1,1]))

h1 = tf.add(tf.matmul(x,w1),b1)
h2 = tf.add(tf.matmul(h1,w2),b2)
func = tf.add(tf.matmul(h2,w3),b3)

cost = tf.reduce_mean(tf.square(func-y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

sess =tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(len(total_x_data)):
    for k in range(1000):
        sess.run(train,feed_dict={x:total_x_data[i],y:total_y_data[i]})

InternalError: Blas GEMM launch failed : a.shape=(373, 7), b.shape=(7, 15), m=373, n=15, k=7
	 [[{{node MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_0_0/_3, Variable/read)]]

Caused by op 'MatMul', defined at:
  File "/home/hadoop/anaconda3/envs/forest/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hadoop/anaconda3/envs/forest/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hadoop/anaconda3/envs/forest/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/hadoop/anaconda3/envs/forest/lib/python3.4/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hadoop/anaconda3/envs/forest/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  

In [ ]:
predict = test_x_data[0]
pre_data = []

input_list = []
print("혈당 값 7개의 입력 : ")
for i in range(7):
    k = input()
    input_list.append(k)

for i,k in enumerate(predict):
    pre = sess.run(func,feed_dict={x:[k]})
    pre_data.append(pre[0])

input_list = np.array(input_list)
predict = sess.run(func,feed_dict={x:[input_list]})[0][0]

if predict - float(input_list[6])>0:
    print("혈당이 높아집니다")
elif predict - float(input_list[6])<0:
    print("혈당이 낮아집니다")
else:
    print("혈당이 같습니다")

print("예측 혈당 : " + str(predict))
    
plt.plot(pre_data,color="red")
#plt.plot(test_y_data[0],color="blue")
plt.show()